# ChatGPT

#### Load documents, build the GPTVectorStoreIndex

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [13]:
# LLM Predictor (gpt-3.5-turbo) + service context
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)

Unknown max input size for gpt-3.5-turbo, using defaults.


In [14]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 27212 tokens
> [build_index_from_nodes] Total embedding token usage: 27212 tokens


#### Query Index

By default, with the help of langchain's PromptSelector abstraction, we use 
a modified refine prompt tailored for ChatGPT-use if the ChatGPT model is used.

In [15]:
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=3,
    streaming=True,
)
response = query_engine.query(
    "What did the author do growing up?", 
)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 8 tokens
> [retrieve] Total embedding token usage: 8 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 0 tokens
> [get_response] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [16]:
response.print_response_stream()

Before college, the author worked on writing and programming. They wrote short stories and tried writing programs on an IBM 1401 using an early version of Fortran. With microcomputers, the author's interest in programming grew, and they eventually created a new dialect of Lisp called Arc. They also started publishing essays online and realized the potential of the internet for publishing written work.

In [17]:
query_engine = index.as_query_engine(
    service_context=service_context,
    similarity_top_k=5,
    streaming=True,
)
response = query_engine.query(
    "What did the author do during his time at RISD?", 
)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
> [retrieve] Total embedding token usage: 12 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 0 tokens
> [get_response] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [18]:
response.print_response_stream()

The author attended RISD and took classes in fundamental subjects like drawing, color, and design. They also learned a lot in the color class they took, but otherwise, they were basically teaching themselves to paint. The author dropped out of RISD in 1993.

**Refine Prompt**: Here is the chat refine prompt 

In [19]:
from llama_index.prompts.chat_prompts import CHAT_REFINE_PROMPT

In [20]:
dict(CHAT_REFINE_PROMPT.prompt)

{'input_variables': ['existing_answer', 'context_msg', 'query_str'],
 'output_parser': None,
 'partial_variables': {},
 'messages': [HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query_str'], output_parser=None, partial_variables={}, template='{query_str}', template_format='f-string', validate_template=True), additional_kwargs={}),
  AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['existing_answer'], output_parser=None, partial_variables={}, template='{existing_answer}', template_format='f-string', validate_template=True), additional_kwargs={}),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context_msg'], output_parser=None, partial_variables={}, template="We have the opportunity to refine the above answer (only if needed) with some more context below.\n------------\n{context_msg}\n------------\nGiven the new context, refine the original answer to better answer the question. If the context isn't useful, output the original answ

#### Query Index (Using the standard Refine Prompt)

If we use the "standard" refine prompt (where the prompt is one text template instead of multiple messages), we find that the results over ChatGPT are worse. 

In [21]:
from llama_index.prompts.default_prompts import DEFAULT_REFINE_PROMPT

In [23]:
query_engine = index.as_query_engine(
    service_context=service_context,
    refine_template=DEFAULT_REFINE_PROMPT,
    similarity_top_k=5,
    streaming=True,
)
response = query_engine.query(
    "What did the author do during his time at RISD?", 
)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
> [retrieve] Total embedding token usage: 12 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 0 tokens
> [get_response] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [25]:
response.print_response_stream()

The author attended RISD and took classes in fundamental subjects like drawing, color, and design. They also learned a lot in the color class they took, but otherwise, they were basically teaching themselves to paint. The author dropped out of RISD in 1993.


### [Beta] Use ChatGPTLLMPredictor

Very simple GPT-Index-native ChatGPT wrapper. Note: this is a beta feature. If this doesn't work please
use the suggested flow above.

In [26]:
# use ChatGPT [beta]
from llama_index.llm_predictor.chatgpt import ChatGPTLLMPredictor
from langchain.prompts.chat import SystemMessagePromptTemplate

llm_predictor = ChatGPTLLMPredictor()

In [27]:
query_engine = index.as_query_engine(
    service_context=service_context,
    streaming=True,
)
response = query_engine.query(
    "What did the author do during his time at RISD?", 
)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens
> [retrieve] Total embedding token usage: 12 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 0 tokens
> [get_response] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [28]:
response.print_response_stream()

The author did the foundation program at RISD, which included classes in fundamental subjects like drawing, color, and design.